# GPS words and fixed points of morphisms

In [2]:
import re
import gpc
import time
import itertools


def writetofile(text):
    with open("out.txt", "a") as ofile:
        ofile.write(str(text) + "\n")


def timing(f):
    """A decorator function timing a functions."""
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('%s function took %0.3f ms' %
              (f.__name__, (time2 - time1) * 1000.0))
        writetofile('%s function took %0.3f ms' %
                    (f.__name__, (time2 - time1) * 1000.0))
        return(ret)
    return wrap

## Substitution

In [3]:
def subs(dic, n):
    """Returns the prefix generated by a morphism starting from 0"""
    s = dic["0"]
    i = 1
    while len(s) < n:
        s = s + dic[s[i]]
        i = i + 1
    return s[:n]

## Patterns we want to avoid in the bi-sequence

In [4]:
# Avoiding only R's in \Theta
cpattern = re.compile("^R+$")
# Avoiding the periodic word 0101010101...
dpattern = re.compile("^(01)+$")
# Avoiding (\Delta, Theta) = (uaaaaaaaa..., vtttttttttt... ), |u|=|v|
epattern = re.compile("((0R){20})|((0E){20})|((1R){20})|((1E){20})$")
# Avoiding some periodic GPS words
fpattern = re.compile(
    "((((0R)+1E){10})|(((0E)+1R){10})|(((1R)+0E){10})|(((1E)+0R){10}))$")

## Testing functions

In [5]:
from multiprocessing import Pool
l = 300


def testword(rule, file=None):
    '''Test if a fixed point of morphism can be a
    generalized pseudostandard word of the required form'''
    dicti = {"0": rule[0], "1": rule[1]}
    # Preparing the prefix of length l
    word = subs(dicti, l)
    # Checking if it can be a GPS word
    ret = gpc.isGPSNaive(word)
    # If so, checking if it contains forbidden patterns
    if (ret[0] == True):
        # Normalizes bi-sequence
        bis = [ret[1], ret[2]]
        # One sequence instead of two
        bis_c = gpc.makeBiseq(ret[1], ret[2])
        # Bi-sequence containing max number of R
        bisR = gpc.maximizeRinBiseq(ret[1], ret[2])
        
        # Avoiding only R's in \Theta,
        # the periodic word 0101010101...
        # and other forbidden forms od (\Delta, \Theta)
        if not (re.match(cpattern, bisR[1])) \
        and not (re.match(dpattern, word)) \
        and not (re.search(epattern, bis_c)) \
        and not (re.search(fpattern, bis_c)):
            return [True, dicti, bis, word[:40]]
        else:
            return None
    else:
        return None


@timing
def getresults(lphi0, lphi1, file=None):
    pool = Pool(processes=4)
    results = []
    for rep1 in range(1, lphi0):
        for rep2 in range(1, lphi1 + 1):
            # Preparing mappings for 0 and 1
            phi0 = ['0' + ''.join(i)
                    for i in itertools.product('01', repeat=rep1)]
            phi1 = [''.join(j) 
                    for j in itertools.product('01', repeat=rep2)]
            # Preparing the set of all couple of rules
            cart = [k for k in itertools.product(phi0, phi1)]
            # Testing all the rules
            r = pool.map_async(testword, cart)
            # Getting only results != None
            filteredresults = list(filter(lambda x: x != None, r.get()))
            if filteredresults != []:
                for res in filteredresults:
                    results.append(res)
    return results

## Results

In [5]:
results = getresults(7,7)
print(results)

getresults function took 111136.812 ms
[[True, {'0': '01', '1': '10'}, ['011111111', 'RERERERER'], '0110100110010110100101100110100110010110'], [True, {'0': '0110', '1': '1001'}, ['011111111', 'RERERERER'], '0110100110010110100101100110100110010110'], [True, {'0': '0110110', '1': '1001001'}, ['011111111', 'RERRERRER'], '0110110100100110010010110110100100110010']]


In [ ]:
print(results)